In [34]:
# Imports
from tweet_processor import generateTweetTensor
from preprocess import genLabels
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from gensim.models import KeyedVectors

from torch.utils.data import DataLoader


from sklearn.model_selection import train_test_split
import models
import pandas as pd
import numpy as np
from torchtext import data



In [8]:

def calculateAcc(a, p):

    b = len(a)
    correct = 0
    for i in range(0, b):
        if a[i] < 0.5  and p[i] < 0.5:
            correct += 1
        elif a[i] > 0.5  and p[i] > 0.5:
            correct += 1

    acc = correct / b
    return acc

#Load Pre-trained model for Words
print("Loading Pre-trained model of many tweets...")
glove = KeyedVectors.load_word2vec_format('glove.twitter.27B.100d.w2vformat.txt')
print("Model loaded.")

# Load tweets from json


Loading Pre-trained model of many tweets...
Model loaded.


In [13]:

model = RNN2(100, 100, 3)
print(model.parameters())
optimizer = torch.optim.Adam(model.parameters(), lr= 0.01)
optimizer.zero_grad()



<generator object Module.parameters at 0x1a30bf59a8>


In [35]:

print("Loading dataset...")
tweets = pd.read_json('trump_tweets_json.json')
tweets = tweets[['created_at', 'text']]
tweets_with_labels = genLabels(tweets)
print("Loaded dataset.")


# Split sets into train, test, and validation
print("Splitting data...")
rest_x, test_x, rest_y, test_y = train_test_split(tweets_with_labels, tweets_with_labels['onehot'], test_size=0.2, random_state=37)
train_x, validate_x, train_y, validate_y = train_test_split(rest_x, rest_y, test_size=0.2, random_state=37)


print("Generating vectors...")
# Generate vector of tweets
loss_fnc = nn.BCEWithLogitsLoss()

train_x["onehot"]

Loading dataset...
Loaded dataset.
Splitting data...
Generating vectors...


2473    [[]]
5707      []
5671    [[]]
7043      []
9320      []
1155      []
1301    [[]]
1570    [[]]
7307    [[]]
7240      []
830       []
2546      []
7216    [[]]
7395      []
2745      []
6970    [[]]
5019    [[]]
349       []
985       []
4300      []
4813    [[]]
3416    [[]]
8557    [[]]
6785      []
3083      []
1706    [[]]
7329    [[]]
5556      []
4907    [[]]
3738      []
        ... 
3798    [[]]
8988      []
7828    [[]]
3269      []
3224      []
432       []
8321      []
453       []
8894      []
5814    [[]]
7184    [[]]
48        []
3194      []
7137    [[]]
6244      []
290     [[]]
2366      []
8802      []
799     [[]]
2589    [[]]
6096      []
1237      []
4380      []
8848      []
3394      []
2659    [[]]
3221      []
6589      []
7074    [[]]
4447      []
Name: onehot, Length: 5987, dtype: object

In [29]:
print("Generating Test Vector...")
test_tweet_vector, test_lengths, test_nulls = generateTweetTensor(glove, test_x)
test_y
#test_y_tensor = torch.LongTensor([x for x in np.array(test_y)])
#test_y_tensor


Generating Test Vector...
              created_at                                               text  \
2232 2019-06-26 23:46:05  ....They came to me asking for help, and I got...   
3067 2019-04-22 18:05:04  Spoke to Prime Minister Ranil Wickremesinghe o...   
3593 2019-02-26 01:50:37  ....This will be remembered as one of the most...   
2776 2019-05-14 20:45:21  It is great to be here in Hackberry, Louisiana...   
3174 2019-04-12 16:38:02  Due to the fact that Democrats are unwilling t...   
6567 2018-04-13 12:01:47  James Comey is a proven LEAKER &amp; LIAR. Vir...   
7503 2017-11-08 03:09:37  The U.S., under my administration, is complete...   
7559 2017-11-02 11:54:36  ...There is also something appropriate about k...   
882  2019-09-25 03:17:40  Sooooo true @LindseyGrahamSC! https://t.co/ZzF...   
934  2019-09-21 21:38:54  �They�re trying to turn what was a Biden scand...   
9141 2017-02-17 21:48:22  The FAKE NEWS media (failing @nytimes, @NBCNew...   
1177 2019-09-05 23:49:22  

2232    [[]]
3067      []
3593      []
2776      []
3174      []
6567    [[]]
7503      []
7559      []
882       []
934     [[]]
9141    [[]]
1177      []
5194      []
6581      []
1163      []
6830    [[]]
6129      []
4983      []
2052    [[]]
8877    [[]]
1470      []
9313      []
6043    [[]]
3275      []
3074      []
1332      []
3046    [[]]
7325    [[]]
6807    [[]]
3778      []
        ... 
3290    [[]]
2160      []
1037      []
6465      []
7231      []
4464      []
6982      []
219     [[]]
3492      []
1013      []
4104    [[]]
4229      []
454       []
2705      []
6359    [[]]
4645    [[]]
2148      []
9237      []
8152      []
5350      []
5576      []
7925      []
7133      []
3679      []
3553      []
7622    [[]]
9041      []
1079    [[]]
7919      []
2891    [[]]
Name: onehot, Length: 1871, dtype: object

In [17]:
for n in range(0, len(test_nulls)):
    i  = test_nulls[n] - n
    test_y_tensor = torch.cat([test_y_tensor[0: i], test_y_tensor[i+1:]])

test_dataset = torch.utils.data.TensorDataset(test_tweet_vector, test_y_tensor, torch.tensor(test_lengths))
test_iter = DataLoader(test_dataset, batch_size=64, shuffle=True, num_workers=0)



NameError: name 'test_y_tensor' is not defined

In [ ]:

print("Generating Validate Vector...")
validate_tweet_vector, validate_lengths, validate_nulls = generateTweetTensor(glove, validate_x)
validate_y_tensor = torch.LongTensor([x for x in np.array(validate_y)])
for n in range(0, len(validate_nulls)):
    i  = validate_nulls[n] - n
    validate_y_tensor = torch.cat([validate_y_tensor[0: i], validate_y_tensor[i+1:]])
val_dataset = torch.utils.data.TensorDataset(validate_tweet_vector, validate_y_tensor, torch.tensor(validate_lengths))
#val_iter = data.BucketIterator(val_dataset, batch_size=64, repeat=False, sort_key=lambda x: len(x.text))
val_iter = DataLoader(val_dataset, batch_size=64, shuffle=True, num_workers=0)


print("Generating Train Vector...")
train_tweet_vector, train_lengths, train_nulls = generateTweetTensor(glove, train_x)
train_y_tensor = torch.LongTensor([x for x in np.array(train_y)])
#actual = (torch.from_numpy(np.eye(10)[label])).float()


for n in range(0, len(train_nulls)):
    i  = train_nulls[n] - n
    train_y_tensor = torch.cat([train_y_tensor[0: i], train_y_tensor[i+1:]])
train_dataset = torch.utils.data.TensorDataset(train_tweet_vector, train_y_tensor, torch.tensor(train_lengths) )
train_iter = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0)


print("Generated buckets.")

# Create rnn
#model = models.RNN(100, 100)
print(model.parameters())
learning_rate = 0.01
num_epochs = 25

# Train models
loss_fnc = nn.BCEWithLogitsLoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
eps = []
training_loss = []
training_accs = []
validation_loss = []
validation_accs = []
print("Model Created.")

for e in range(num_epochs):
    eps += [e]
    print("Epoch: " + str(e))
    for j, batch in enumerate(train_iter):
        inputs = batch[0]
        actual = batch[1]
        lengths_float = batch[2]
        lengths = lengths_float.long()
        print("success")
        optimizer.zero_grad()
        predicted = model(inputs,lengths)
        loss = loss_fnc(predicted, actual.float())
        loss.backward()
        optimizer.step()

    print("Calculating Training Accuracy...")
    # Calculate Training Accuracy
    train_labels = []
    train_preds = []


    for k, t_batch in enumerate(train_iter):
        t_inputs = t_batch[0]
        t_actual = t_batch[1]
        t_lengths_floats = t_batch[2]
        t_lengths = t_lengths_floats.long()

        train_labels += t_actual.tolist()
        t_predicted = model(t_inputs, t_lengths)
        train_preds += t_predicted.tolist()

    t_acc = calculateAcc(train_labels, train_preds)
    t_loss = loss_fnc(torch.FloatTensor(train_preds), torch.FloatTensor(train_labels))
    training_accs += [t_acc]
    training_loss += [t_loss.item()]
    print("Training Accuracy: " + str(t_acc))
    print("Training Loss: " + str(t_loss.item()))

    # Calculate Validation Accuracy
    valid_labels = []
    valid_preds = []
    for k, v_batch in enumerate(val_iter):
        v_inputs = v_batch[0]
        v_actual = v_batch[1]
        v_lengths_floats = v_batch[2]
        v_lengths = v_lengths_floats.long()
        valid_labels += v_actual.tolist()
        v_predicted = model(v_inputs, v_lengths)
        valid_preds += v_predicted.tolist()

    v_acc = calculateAcc(valid_labels, valid_preds)
    v_loss = loss_fnc(torch.FloatTensor(valid_preds), torch.FloatTensor(valid_labels))
    validation_accs += [v_acc]
    validation_loss += [v_loss.item()]
    print("Validation Accuracy: " + str(v_acc))
    print("Validation Loss: " + str(v_loss.item()))

# Calculate Testing Accuracy
test_labels = []
test_preds = []
for k, t_batch in enumerate(test_iter):
    t_inputs = t_batch[0]
    t_actual = t_batch[1]
    t_lengths_floats = t_batch[2]
    t_lengths = t_lengths_floats.long()
    test_labels += t_actual.tolist()
    t_predicted = model(t_inputs, t_lengths)
    test_preds += t_predicted.tolist()

test_loss = loss_fnc(torch.FloatTensor(test_preds), torch.FloatTensor(test_labels))
print("TESTING LOSS: " + str(test_loss.item()))
t_acc = calculateAcc(test_labels, test_preds)
print("TESTING ACCURACY: " + str(t_acc))

#   Display Accuracy vs. Epoch
fig, ax = plt.subplots()
ax.plot(eps, training_accs, label='Training Data')
ax.plot(eps, validation_accs, label='Validation Data')
ax.set(xlabel='Number of Epochs', ylabel='Accuracy', title='Accuracy vs. Epoch')
ax.set_ylim(0, 1)
ax.grid()
ax.legend()
plt.show()

#   Display Loss vs. Epoch
fig, ax = plt.subplots()
ax.plot(eps, training_loss, label='Training Data')
ax.plot(eps, validation_loss, label='Validation Data')
ax.set(xlabel='Number of Epochs', ylabel='Loss', title='Loss vs. Epoch')
ax.set_ylim(0, 1)
ax.grid()
ax.legend()
plt.show()

In [12]:

class RNN2(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN2, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

def forward(self, data, length, hidden):
        length = lengths.cpu()
        data = torch.transpose(data, 0, 1)
        x = torch.nn.utils.rnn.pack_padded_sequence(data, lengths= length, enforce_sorted = False) # unpad
        combined = torch.cat((embeds.view(1, -1), hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden
